Cell for notes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import sys
sys.path.append(os.path.abspath('../input/efficientnet/efficientnet-master/efficientnet-master/'))
from efficientnet import EfficientNetB5
from sklearn.utils import class_weight, shuffle
from keras import models
from keras.models import Model
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from keras.optimizers import Adam, Optimizer
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from keras.utils import Sequence, to_categorical
import keras.backend as K
from keras.legacy import interfaces
import tensorflow as tf
from numpy.random import seed
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from imgaug import augmenters as iaa
seed(1)
import warnings
warnings.filterwarnings("ignore")
import cv2
import zipfile

# Loading the Data and preprocessing the images

I used images from the 2019 competetion as well as the <a href='https://www.kaggle.com/c/diabetic-retinopathy-detection/data'> previous competition from 2015 </a>. To keep the classes somewhat balanced, I'm only using 700 images from each class from the 2015 dataset. I did preprocessed the images locally by resizing every image to $(456,456)$, which is <a href='https://github.com/qubvel/efficientnet/blob/9e4c560bd06da3fd90c6aeee51b48da22f2318b7/efficientnet/model.py#L551'>EfficientNetB5's native image size</a>.

## Notes
<ul>
    <li> I did not crop the blackspace out of the training images at any point of this competition. <a href='https://www.kaggle.com/taindow/be-careful-what-you-train-on'> Per Tom Aindow's work</a>, there is an unwanted correlation between the black space and labels in the 2015 dataset that is not present in the 2019 public test set. However, any attempt at cropping consistently led to lower LB scores and lower validation scores during training. For reference, the cropping method I tried was similar to the circle cropping method found <a href='https://www.kaggle.com/taindow/pre-processing-train-and-test-images'>here</a>.</li>
    <li>For most of the competition, I kwas using an image size of $(224,224)$. Changing the image size to $(456,456)$ was my single biggest jump in LB score.</li>
</ul>    

In [ ]:
TRAIN_PATH_NEW = '../input/upload3/train_images/train_images/'
TRAIN_PATH_OLD = '../input/upload3/train_2015/train_2015/'
TEST_PATH = '../input/aptos2019-blindness-detection/test_images/'

df_old = pd.read_csv('../input/upload2/trainlabels_2015.csv')
df_old.columns = ['id_code', 'diagnosis']
df_old['id_code'] = TRAIN_PATH_OLD + df_old['id_code'] + '.jpeg'

df_new = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')

df_new['id_code'] = TRAIN_PATH_NEW + df_new['id_code'] + ".png"

df = pd.concat([df_old, df_new], axis = 0)
df_old['diagnosis'] = df_old['diagnosis'].astype(str)
df_new['diagnosis'] = df_new['diagnosis'].astype(str)

train, test = train_test_split(df_new, test_size = .2, random_state = 123, stratify = df_new['diagnosis'])

batch_size = 4
image_size = (456,456)

figax, ax = plt.subplots(1,1,figsize = (10,5))
sns.barplot(x = df['diagnosis'].value_counts().index, y = df['diagnosis'].value_counts()).set_title("Diagnosis Distribution")
sns.despine(bottom = True, left = True)
sns.set_style('dark')
ax.set_ylabel("")
ax.get_yaxis().set_visible(False)
plt.show()

# Exploring the images

Since there is a large amount of variability in the images (e.g., brightness and contrast), we need to do some image preprocessing before training a model. Like many other participants in this competition, I used Ben Graham's preprocessing method described in <a href='https://www.kaggle.com/ratthachat/aptos-updatedv14-preprocessing-ben-s-cropping'> this kernel</a>. As mentioned above, I also considered cropping blackspace while preprocessing the images, but ultimately I decided against it. Huge shoutout to <a href='https://www.kaggle.com/ratthachat'> @ratthachat </a> for giving so many of us a place to start in this competition.

In [ ]:
fig = plt.figure(figsize = (16,16))
# Test image preprocessing function
for i in range(6):
    img = cv2.imread(df.iloc[i,:]['id_code'])
    img = cv2.resize(img, image_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    mask = gray > 10
    img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
    img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
    img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    
    crop = np.stack([img1,img2,img3],axis=-1)
    crop_transform = cv2.addWeighted (crop,4, cv2.GaussianBlur(crop , (0,0) ,10) ,-4 ,128)
    crop = cv2.resize(crop, image_size)
    crop_transform = cv2.resize(crop_transform, image_size)
    ax = fig.add_subplot(4,6,i+1)
    plt.imshow(img)
    ax2 = fig.add_subplot(4,6,i+7)
    plt.imshow(gray, cmap='gray')
    ax3 = fig.add_subplot(4,6,i+13)
    plt.imshow(crop)
    ax4 = fig.add_subplot(4,6,i+19)
    plt.imshow(crop_transform)

# Image generator and augmentations



In [ ]:
class ImgGenerator(Sequence):
    ########################################################################################
    ## Generic custom Generator.
    ## Parameters:
    ##     df (req): Pandas DataFrame
    ##     x_col (req): Column name with filenames
    ##     y_col (req): Column with class labels
    ##     file_path (opt): File Path
    ##     img_extension (opt): image file extension (if not included in df[x_col])
    ##     img_size (opt): Image size, defautls 320 x 320
    ##     batch_size (opt): batch_size. defaults to 32
    ##     should_shuffle (opt): If True, shuffles df after each epoch.  Defaults True
    ##     should_aug (opt): If True, augments images.
    ##     preprocessing_function (opt): function to call on each image before augmentation
    ##     postprocessing_function (opt): function to call on each image after augmentation
    ##     iaa_seq (opt): augmentation sequence to run on each image
    ########################################################################################
    def __init__(self,
                 df = None,
                 x_col = None,
                 y_col = None,
                 num_classes = None,
                 file_path = "",
                 img_extension = "",
                 img_size = (320,320),
                 batch_size = 32,
                 should_shuffle = True,
                 should_aug = False,
                 preprocessing_function = None,
                 postprocessing_function = None,
                 should_mix = True,
                 iaa_seq = None):
        self.df = df
        self.x_col = x_col
        self.y_col = y_col
        self.num_classes = num_classes
        self.file_path = file_path
        self.img_extension = img_extension
        self.img_size = img_size
        self.batch_size = batch_size
        self.should_shuffle = should_shuffle
        self.should_aug = should_aug
        self.preprocessing_function = preprocessing_function
        self.postprocessing_function = postprocessing_function
        self.iaa_seq = iaa_seq
        self.should_mix = should_mix
    
    def __len__(self):
        return int(np.ceil(self.df.shape[0] / float(self.batch_size)))
   
    def __getitem__(self, idx):
        batch_x = self.df[self.x_col][idx * self.batch_size:(idx + 1) * self.batch_size]
        if self.y_col is not None:
            if self.num_classes is not None:
                batch_y = to_categorical(self.df[self.y_col][idx * self.batch_size:(idx + 1) * self.batch_size], num_classes = self.num_classes)
            else:
                batch_y = self.df[self.y_col][idx * self.batch_size:(idx + 1) * self.batch_size]
        else:
            batch_y = None
        return self.generate_images(batch_x, batch_y, self.should_aug)
        
    def on_epoch_end(self):
        if self.should_shuffle:
            self.df = self.df.sample(frac = 1)
    
    
    # https://arxiv.org/pdf/1710.09412.pdf
    def mix_up(self, x, y):
        lam = np.random.beta(0.2, 0.4)
        ori_index = np.arange(int(len(x)))
        index_array = np.arange(int(len(x)))
        np.random.shuffle(index_array)        
        
        mixed_x = lam * x[ori_index] + (1 - lam) * x[index_array]
        mixed_y = lam * y[ori_index] + (1 - lam) * y[index_array]
        
        return mixed_x, mixed_y
    
    def generate_images(self, batch_x, batch_y, should_aug):
        batch_images = []
        if batch_y is None:
            for (file) in batch_x:
                if self.img_extension != "":
                    full_path = os.path.join(self.file_path, file + "." + self.img_extension)
                else:
                    full_path = os.path.join(self.file_path, file)
                img = cv2.imread(full_path)
                img = cv2.resize(img, self.img_size)
            
                # Run preprocessing function
                if self.preprocessing_function is not None:
                    img = self.preprocessing_function(img)
                
                # Run post processing function    
                if self.postprocessing_function is not None:
                    img = self.postprocessing_function(img)
                batch_images.append(img)
            batch_images = np.array(batch_images, np.float32) / 255
            return batch_images
                
        for (file, label) in zip(batch_x, batch_y):
            # Read and resize
            if self.img_extension != "":
                full_path = os.path.join(self.file_path, file + "." + self.img_extension)
            else:
                full_path = os.path.join(self.file_path, file)
            img = cv2.imread(full_path)
            
            
            # Run preprocessing function
            if self.preprocessing_function is not None:
                img = self.preprocessing_function(img)
                
            # Augment as needed
            if self.should_aug:
                img = self.iaa_seq.augment_image(img)
                
            # Run post processing function    
            if self.postprocessing_function is not None:
                img = self.postprocessing_function(img)
            img = cv2.resize(img, self.img_size)
            batch_images.append(img)
            
        # Scale by default (not configurable)
        batch_images = np.array(batch_images, np.float32) / 255
        batch_y = np.array(batch_y, np.float32)
        if self.should_mix:
            batch_images, batch_y = self.mix_up(batch_images, batch_y)
        return batch_images, batch_y

In [ ]:
# Preprocessing function to crop blackspace before augmentation.
# postprocess_input applies Ben's preprocessing function after augmentation
# See: https://www.kaggle.com/ratthachat/aptos-updatedv14-preprocessing-ben-s-cropping

def preprocess_input(x):
    gray = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
    mask = gray > 7
    if x[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0] > 0:
        img1=x[:,:,0][np.ix_(mask.any(1),mask.any(0))]
        img2=x[:,:,1][np.ix_(mask.any(1),mask.any(0))]
        img3=x[:,:,2][np.ix_(mask.any(1),mask.any(0))]
        x = np.stack([img1,img2,img3],axis=-1)
    return x

def postprocess_input(x):
    return cv2.addWeighted(x,4, cv2.GaussianBlur( x , (0,0) ,10) ,-4 ,128)

# Sequential image augmentor.
# I used horizontal flips, Gaussian Blurring, Gaussian Noise, and affine transformations.

seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Sometimes(0.5,iaa.GaussianBlur(sigma=(0, 0.5))),
    iaa.ContrastNormalization((0.9, 1.2)),
    iaa.Sometimes(0.5,iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)),
    iaa.Sometimes(0.5,iaa.Multiply((0.8, 1.2), per_channel=0.2)),
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        rotate=(-120, 120),
        shear=(-15, 15)
    )
], random_order=True)

In [ ]:
pretrain_generator = ImgGenerator(df = df_old,
                                  x_col = "id_code",
                                  y_col = 'diagnosis',
                                  batch_size = batch_size,
                                  #num_classes = 5,
                                  img_size = image_size,
                                  #preprocessing_function = preprocess_input,
                                  postprocessing_function = postprocess_input,
                                  should_aug = True,
                                  should_mix = False,
                                  iaa_seq = seq)
pretrain_validation_generator = ImgGenerator(df = df_new,
                                             x_col = "id_code",
                                             y_col = 'diagnosis',
                                             batch_size = batch_size,
                                             #num_classes = 5,
                                             img_size = image_size,
                                             #preprocessing_function = preprocess_input,
                                             postprocessing_function = postprocess_input,
                                             should_aug = False,
                                             should_mix = False)

train_generator = ImgGenerator(df = train,
                                  x_col = "id_code",
                                  y_col = 'diagnosis',
                                  batch_size = batch_size,
                                  #num_classes = 5,
                                  img_size = image_size,
                                  #preprocessing_function = preprocess_input,
                                  postprocessing_function = postprocess_input,
                                  should_aug = True,
                                  should_mix = False,
                                  iaa_seq = seq)

validation_generator = ImgGenerator(df = test,
                                      x_col = "id_code",
                                      y_col = 'diagnosis',
                                      batch_size = batch_size,
                                      #num_classes = 5,
                                      img_size = image_size,
                                      #preprocessing_function = preprocess_input,
                                      postprocessing_function = postprocess_input,
                                      should_aug = False,
                                      should_mix = False)

# Accumulating Adam Optimizer
Since the model we're using is memory intestive, we are unable to use a large batch size during training. This is problematic since small batch sizes can lead to overfitting. To handle this problem, we define an accumulating Adam Optimizer. This optimizer will process several batches before updating the weights of the model. <a href='Source https://github.com/keras-team/keras/issues/3556'>Source</a>

In [ ]:
class AdamAccumulate(Optimizer):

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., amsgrad=False, accum_iters=1, **kwargs):
        if accum_iters < 1:
            raise ValueError('accum_iters must be >= 1')
        super(AdamAccumulate, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsgrad = amsgrad
        self.accum_iters = K.variable(accum_iters, K.dtype(self.iterations))
        self.accum_iters_float = K.cast(self.accum_iters, K.floatx())

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr

        completed_updates = K.cast(K.tf.floordiv(self.iterations, self.accum_iters), K.floatx())

        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * completed_updates))

        t = completed_updates + 1

        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) / (1. - K.pow(self.beta_1, t)))

        # self.iterations incremented after processing a batch
        # batch:              1 2 3 4 5 6 7 8 9
        # self.iterations:    0 1 2 3 4 5 6 7 8
        # update_switch = 1:        x       x    (if accum_iters=4)  
        update_switch = K.equal((self.iterations + 1) % self.accum_iters, 0)
        update_switch = K.cast(update_switch, K.floatx())

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        gs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]

        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]

        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat, tg in zip(params, grads, ms, vs, vhats, gs):

            sum_grad = tg + g
            avg_grad = sum_grad / self.accum_iters_float

            m_t = (self.beta_1 * m) + (1. - self.beta_1) * avg_grad
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(avg_grad)

            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                p_t = p - lr_t * m_t / (K.sqrt(vhat_t) + self.epsilon)
                self.updates.append(K.update(vhat, (1 - update_switch) * vhat + update_switch * vhat_t))
            else:
                p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon)

            self.updates.append(K.update(m, (1 - update_switch) * m + update_switch * m_t))
            self.updates.append(K.update(v, (1 - update_switch) * v + update_switch * v_t))
            self.updates.append(K.update(tg, (1 - update_switch) * sum_grad))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, (1 - update_switch) * p + update_switch * new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon,
                  'amsgrad': self.amsgrad}
        base_config = super(AdamAccumulate, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

# Custom Callback to evaluate quadratic weighted kappa
This competition is scored using <a href='https://en.wikipedia.org/wiki/Cohen%27s_kappa#Weighted_kappa'>quadratic weighted Kappa</a>. It is not easy to implement quadratic weighted kappa as an evaluation metric. As a workaround, we calculate the quadratic weighted kappa after each epoch <a href='https://stackoverflow.com/questions/37657260/how-to-implement-custom-metric-in-keras
'>as a callback</a>. This callback object also stores a history of quadratic weighted kappa, confusion matrices, and value_counts DataFrames for each epoch.

## Notes:
<ul>
    <li>Once I switched to a larger image size, I stopped using this callback since it increases the training time by about sixty seconds per epoch. </li>
    <li>Quadratic weighted kappa turned out to be volatile during training. It was much smoother to use <a href='https://en.wikipedia.org/wiki/Mean_squared_error'>mean squared error</a> as a loss function.</li>

In [ ]:
class EvaluationMetrics(Callback):
    def __init__(self, validation_data = None, batch_size = 32):
        super(Callback, self).__init__()
        self.validation_generator = validation_data
        self.batch_size = batch_size
    
    def on_train_begin(self, logs={}):
        self._val_qwk = []
        self._val_pred_counts = pd.DataFrame(index = [0,1,2,3,4])
        self._val_confusion_matrices = []
        self._epoch = 1

    def on_epoch_end(self, batch, logs={}):
        y_val = np.array([])
        for i in range(len(validation_generator)):
            y_val_batch = validation_generator[i][1]
            y_val = np.append(y_val, y_val_batch)
            y_val.astype(int)
        y_predict = model.predict_generator(generator=validation_generator,
                                            steps=np.ceil(float(len(y_val)) / float(batch_size)),
                                            verbose = 1).tolist()

        for i in range(len(y_predict)):
            if y_predict[i][0] <= 0.5:
                y_predict[i][0] = 0
            elif y_predict[i][0] <= 1.5:
                y_predict[i][0] = 1
            elif y_predict[i][0] <= 2.5:
                y_predict[i][0] = 2
            elif y_predict[i][0] <= 3.5:
                y_predict[i][0] = 3
            else:
                y_predict[i][0] = 4
        
        y_predict = np.array(y_predict)
        
        self.qwk_hx(y_val, y_predict)
        self.value_counts_hx(y_predict)
        self.confusion_matrix_hx(y_val, y_predict)
        self._epoch += 1
        return
    
    def qwk_hx(self, y_val, y_predict):
        qwk = cohen_kappa_score(y_val,
                                y_predict,
                                labels = [0,1,2,3,4],
                                weights='quadratic')
        print("\nquadratic weighted kappa score: {0}\n".format(qwk))
        self._val_qwk.append(qwk)
        return
        
    def value_counts_hx(self, y_predict):
        self._val_pred_counts['Epoch {0}'.format(self._epoch)] = pd.DataFrame(y_predict)[0].value_counts()
        self._val_pred_counts.fillna(0,inplace=True)
        self._val_pred_counts = self._val_pred_counts.astype(int)
        print("Value counts at Epoch {0}:\n".format(self._epoch))
        print(self._val_pred_counts.head())
        return
    
    def confusion_matrix_hx(self, y_val, y_predict):
        self._val_confusion_matrices.append(confusion_matrix(y_val, y_predict))
        return

    def get_qwk_hx(self):
        return self._val_qwk
    
    def get_value_counts_hx(self):
        return self._val_pred_counts
    
    def get_confusion_matrix_hx(self):
        return self.val_confusion_matrices

eval_metrics = EvaluationMetrics(validation_data = validation_generator, batch_size = batch_size)

# Define and train the model
The model itself is fairly simple. I'm using EfficientNetB5 with weights pretrained on the imagenet dataset. The model was trained and validated using the 2019 images mixed with 3,500 images from the 2015 dataset (700 from each class)

In [ ]:
def mse (y_true, y_pred):
    return K.mean(K.square(y_pred -y_true), axis=-1)

inputs = Input((456, 456, 3))
base_model = EfficientNetB5(include_top=False, weights=None, input_tensor = inputs)
base_model.load_weights('../input/efficientnet-keras-weights-b0b5/efficientnet-b5_imagenet_1000_notop.h5')
out = GlobalAveragePooling2D()(base_model.output)
out = Dropout(0.5)(out)
out = Dense(1024,activation="relu", kernel_regularizer=l2(0.01))(out)
out = Dropout(0.5)(out)
out = Dense(1,activation="relu", kernel_regularizer=l2(0.01))(out)
model = Model(inputs, out)
# Define Callsbacks
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0000001)

early_stopping = EarlyStopping(monitor = 'val_loss',
                               patience = 10,
                               restore_best_weights = True)

# Skipping the training in this kernel to speed up commit time.
'''
# Begin model warmup
for layer in model.layers[:-5]:
    layer.trainable = False
    
model.compile(loss='mse', optimizer=AdamAccumulate(lr=0.001, accum_iters = 4), metrics=[mse])

print("\nWarming up model: " + "\n"*2)
epochs = 5
history = model.fit_generator(pretrain_generator,
                              epochs=epochs,
                              steps_per_epoch = (df_old.shape[0] // batch_size) + 1,
                              validation_data = pretrain_validation_generator,
                              validation_steps = (df_new.shape[0] // batch_size) + 1,
                              workers = 2,
                              #callbacks = [eval_metrics],
                              use_multiprocessing = True,
                              verbose=1)

# Train all layers
for layer in model.layers:
    layer.trainable = True

print('\nBegining training session: ' + '\n'*2)
model.compile(loss='mse', optimizer=AdamAccumulate(lr=0.0005, accum_iters = 6), metrics=[mse])

epochs = 10
history2 = model.fit_generator(pretrain_generator,
                              epochs=epochs,
                              steps_per_epoch = (df_old.shape[0] // batch_size) + 1,
                              validation_data = pretrain_validation_generator,
                              validation_steps = (df_new.shape[0] // batch_size) + 1,
                              callbacks=[learning_rate_reduction, early_stopping],# eval_metrics],
                              workers = 2,
                              use_multiprocessing = True,
                              verbose=1)

epochs = 40
history3 = model.fit_generator(train_generator,
                              epochs=epochs,
                              steps_per_epoch = (train.shape[0] // batch_size) + 1,
                              validation_data = validation_generator,
                              validation_steps = (test.shape[0] // batch_size) + 1,
                              callbacks=[learning_rate_reduction, early_stopping],# eval_metrics],
                              workers = 2,
                              use_multiprocessing = True,
                              verbose=1)
model.save('model.h5')
'''

# Test Time Augmentation for final submission

For the final submission, I used 6-fold TTA using flips, affine transformations, and rotations of up to $90^\circ$. The model predicted the class on the original image, as well as five augmented images. Then they voted on the label for the final submission. Note: in the event of a tie the model chose the lower label. This was not intentional, I forgot to fix it in time.


## Evaluate predicted quadratic weighted kappa on validation set without TTA

In [ ]:
y_val = np.array([])
for i in range(len(validation_generator)):
    v_val_batch = validation_generator[i][1]
    y_val = np.append(y_val, v_val_batch)
y_val.astype(int)
y_pred = model.predict_generator(generator=validation_generator,
                                 steps=np.ceil(float(len(y_val)) / float(batch_size)),
                                 verbose=1)

y_pred[y_pred <= 0.5] = 0
y_pred[(y_pred <= 1.5) & (y_pred > 0.5)] = 1
y_pred[(y_pred <= 2.5) & (y_pred > 1.5)] = 2
y_pred[(y_pred <= 3.5) & (y_pred > 2.5)] = 3
y_pred[y_pred > 3.5] = 4

score = cohen_kappa_score(y_val,
                          y_pred.astype(int),
                          labels=[0,1,2,3,4],
                          weights='quadratic')

print("Quadratic weighted kappa: {0}".format(score))

## Evaluate predicted quadratic weighted kappa on validation set with TTA

In [ ]:
seq_tta = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    #iaa.Sometimes(0.5,iaa.GaussianBlur(sigma=(0, 0.5))),
    #iaa.ContrastNormalization((0.9, 1.2)),
    #iaa.Sometimes(0.5,iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)),
    #iaa.Sometimes(0.5,iaa.Multiply((0.8, 1.2), per_channel=0.2)),
    iaa.Affine(
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        translate_percent={"x": (-0.05, 0.05), "y": (-0.05, 0.05)},
        rotate=(-90, 90),
        shear=(-15, 15)
    )
], random_order=True)

tta_df = test.copy()

preds = []
for ix, idx in enumerate(tta_df['id_code']):
    image = cv2.imread(idx)
    image = cv2.resize(image, (456,456))
    score_ary = [0 for i in range(5)]
    for i in range(6):
        if i == 0:
            score_predict = model.predict((postprocess_input(image)[np.newaxis])/255)
        else:
            score_predict = model.predict((postprocess_input(seq_tta.augment_image(image))[np.newaxis])/255) 
        if score_predict <= 0.5:
            score_predict = 0
        elif score_predict <= 1.5:
            score_predict = 1
        elif score_predict <= 2.5:
            score_predict = 2
        elif score_predict <= 3.5:
            score_predict = 3
        else:
            score_predict = 4
        score_ary[score_predict] += 1
    score_ary = np.array(score_ary)
    label = np.argmax(score_ary)
    preds.append(label)    

tta_df['diagnosis'] = np.array(preds, np.int)
tta_df['id_code'] = tta_df['id_code'].str.split(".png", expand = True)[0]

score = cohen_kappa_score(test['diagnosis'].astype(int),
                         list(tta_df['diagnosis']),
                          labels=[0,1,2,3,4],
                          weights='quadratic')
  
print("TTA score: {0}".format(score))

# Final submission with TTA

In [ ]:
sub_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
sub_df['id_code'] = sub_df['id_code']+'.png'

preds = []
for ix, idx in enumerate(sub_df['id_code']):
    path = '../input/aptos2019-blindness-detection/test_images/' + idx
    image = cv2.imread(path)
    image = cv2.resize(image, (456,456))
    score_predict = [model.predict((postprocess_input(seq_tta.augment_image(image))[np.newaxis])/255) for i in range(5)]
    score_predict.append(model.predict((postprocess_input(image)[np.newaxis])/255))
    score_predict = sum(score_predict) / float(len(score_predict))
    if score_predict <= 0.5:
        score_predict = 0
    elif score_predict <= 1.5:
        score_predict = 1
    elif score_predict <= 2.5:
        score_predict = 2
    elif score_predict <= 3.5:
        score_predict = 3
    else:
        score_predict = 4
    preds.append(int(score_predict))

sub_df['diagnosis'] = np.array(preds, np.int)
sub_df['id_code'] = sub_df['id_code'].str.split(".png", expand = True)[0]
sub_df.to_csv('submission.csv', index = False)

sub_df.head()

In [ ]:
figax, ax = plt.subplots(1,1,figsize = (10,5))
sns.barplot(x = sub_df['diagnosis'].value_counts().index, y = sub_df['diagnosis'].value_counts()).set_title("Predicted value distribution")
sns.despine(bottom = True, left = True)
sns.set_style('dark')
ax.set_ylabel("")
ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
fig = plt.figure(figsize = (30,10))
ax = fig.add_subplot(1,2,1)
sns.lineplot(x = range(1, len(history2.history['loss']) + 1), y = history2.history['loss'], ax = ax, label = 'Training Mean Squared Error')
sns.lineplot(x = range(1, len(history2.history['val_loss']) + 1), y = history2.history['val_loss'], ax = ax, label = 'Validation Mean Squared Error')
ax.set_ylabel("")
ax.set_yticks([0,0.5,1,1.5])
ax.set_xlabel("Epoch")
ax.set_xticks(range(1,len(history2.history['val_loss']) + 1))
ax.set_title("Pre-Training History")
plt.show()

In [ ]:
fig = plt.figure(figsize = (30,10))
ax = fig.add_subplot(1,2,1)
sns.lineplot(x = range(1, len(history3.history['loss']) + 1), y = history3.history['loss'], ax = ax, label = 'Training Mean Squared Error')
sns.lineplot(x = range(1, len(history3.history['val_loss']) + 1), y = history3.history['val_loss'], ax = ax, label = 'Validation Mean Squared Error')
ax.set_ylabel("")
ax.set_yticks([0, 0.25, 0.5, 0.75, 1])
ax.set_xlabel("Epoch")
ax.set_xticks(range(1,len(history3.history['val_loss']) + 1))
ax.set_title("Training History")
plt.show()